In [180]:
import pandas as pd
import requests
from datetime import datetime
import numpy as np
from IPython.display import display

### Outline:

Trying to predict the next 5 years of GHG emissions given 1 year of GHG emissions data (scope 1, scope 2, and scope 3) and business growth data. Here is the basic framework:

1. Input data: 
    - 2 years of revenue/sales or assets data if the company is pre-revenue (int)
    - 1 years worth of GHG emissions data: total scope 1, total scope 2, and total scope 3. (float)
    - Additional variables: number of employees (int), company industry (str), total assets (int), stock price (float)
2. Calculate the growth rate using the 2 years of revenue/sales data
3. Create a growth model and forecast the expected growth rate for the next 5 years 
4. Create a tree ensemble model for each scope category and predict the expected scope emissions for each year based on the forecasted growth rate

- How to incorporate target/expected growth rates?
- How to incorporate additional variables such as number of employees, assets, etc.
- How to incorporate macro/micro trends
- How to incorporate AI/ML models

In [6]:
def basic_growth_model(value_year1, value_year2, num_years):
    value_growth_rate = (value_year2 - value_year1) / value_year1  # Calculate annual growth rate
    forecasted_values = [value_year2]

    for year in range(1, num_years + 1):
        adjusted_value_growth_rate = value_growth_rate * (1 - 0.1 * year)  # Calculate adjusted growth rate
        next_value = forecasted_values[-1] * (1 + adjusted_value_growth_rate)  # Forecast next year's value
        forecasted_values.append(next_value)

    return forecasted_values[1:]


def forecast_growth(num_years, financial_data):
    forecasted_sales = basic_growth_model(financial_data['sales_year1'], financial_data['sales_year2'], num_years)
    forecasted_revenue = basic_growth_model(financial_data['revenue_year1'], financial_data['revenue_year2'], num_years)
    return forecasted_sales, forecasted_revenue


In [7]:
# Example usage
example_financial_data = {
    'sales_year1': 1000,
    'sales_year2': 1200,
    'revenue_year1': 2000,
    'revenue_year2': 2400
}

forecasted_sales, forecasted_revenue = forecast_growth(5, example_financial_data)

print("Forecasted Sales for the next 5 years:", forecasted_sales)
print("Forecasted Revenue for the next 5 years:", forecasted_revenue)

Forecasted Sales for the next 5 years: [1416.0, 1642.5600000000002, 1872.5184, 2097.220608, 2306.9426688000003]
Forecasted Revenue for the next 5 years: [2832.0, 3285.1200000000003, 3745.0368, 4194.441216, 4613.885337600001]


In [96]:
df_2022 = pd.read_csv('../external_datasets/top_250_emissions/original/Public profit and emission database - 2022.csv')

df_2022.columns

cols_2022 = ['Company Name','2022 Scope 1 emissions tons CO₂e', '2022 Scope 2 emissions tons CO₂e',
       '2022 Scope 3 emissions tons CO₂e','2022 Sales / Revenue (Millions USD)', '2022 Profit (Millions USD)',
       '2021-2022 Datasource emission data']

cols_mapping_2022 = {'Company Name': 'company_name',
                     '2022 Scope 1 emissions tons CO₂e': 'scope_1', 
                     '2022 Scope 2 emissions tons CO₂e': 'scope_2',
                     '2022 Scope 3 emissions tons CO₂e': 'scope_3',
                     '2022 Sales / Revenue (Millions USD)': 'sales_revenue', 
                     '2022 Profit (Millions USD)': 'profit',
                     '2021-2022 Datasource emission data': 'data_source'}

df_2022_clean = df_2022[cols_2022]
df_2022_clean = df_2022_clean.rename(columns=cols_mapping_2022)
df_2022_clean['year'] = 2022
df_2022_clean['company_name'] = df_2022_clean['company_name'].str.strip()
df_2022_clean

,company_name,scope_1,scope_2,scope_3,sales_revenue,profit,data_source,year
0,JPMorgan Chase,88 553,783 616,156 845,$179 930,$41 800,page 1: https://www.jpmorganchase.com/content/...,2022
1,Saudi Arabian Oil Company (Saudi Aramco),55 700 000,16 100 000,NaN,$589 470,$156 360,"page 63, https://www.aramco.com/-/media/public...",2022
2,ICBC,97 400,1 923 900,NaN,$216 770,$52 470,"page 27, https://v.icbc.com.cn/userfiles/Resou...",2022
3,China Construction Bank,69 497,1 613 315,NaN,$203 080,$48 250,"page 52, http://group.ccb.com/eng/attachDir/20...",2022
4,Agricultural Bank of China,222 353,1 931 555,NaN,$186 140,$37 920,"page 98, https://www.abchina.com/en/aboutus/cs...",2022
...,...,...,...,...,...,...,...,...
203,NVIDIA,4 612,133 569,2 701 477,$26 970,$4 370,"page no. 51, CSR 2022, https://images.nvidia.c...",2022
204,3M,2 420 000,1 460 000,11 636 000,$33 430,$5 450,"page 149, https://multimedia.3m.com/mws/media/...",2022
205,EOG Resources,510 000,400 000,110 300 000,$28 430,$9 390,"page no. 4, https://eogresources-com.s3.us-wes...",2022
206,Takeda Pharmaceutical,277 000,291 000,5 166 000,$29 990,$2 090,NaN,2022


In [98]:
cik_list = []
for company_name in df_2022_clean['company_name']:
    formatted_company_name = company_name.strip(' ').replace(' ', '%20')

    response_obj = requests.get(f'https://financialmodelingprep.com/api/v3/cik-search/{formatted_company_name}?apikey=SWGIUgFhzgYfPKE5F9NsDyPX3SYTb72k')
    json_obj = response_obj.json()

    while type(json_obj) == list:
        if len(json_obj) > 0:
            json_obj = json_obj[0]
        else:
            break

    if type(json_obj) == dict:
        json_obj['original_name'] = company_name.strip(' ')

    cik_list.append(json_obj)
    print(json_obj)


{'cik': '0000019617', 'name': 'JPMORGAN CHASE & CO', 'original_name': 'JPMorgan Chase'}
[]
{'cik': '0000917358', 'name': 'ICBC STANDARD SECURITIES INC.', 'original_name': 'ICBC'}
[]
[]
{'cik': '0000070858', 'name': 'BANK OF AMERICA CORP /DE/', 'original_name': 'Bank of America'}
{'cik': '0001413936', 'name': 'ALPHABET PARTNERS LP', 'original_name': 'Alphabet'}
{'cik': '0001082970', 'name': 'EXXONMOBIL INVESTMENT MANAGEMENT INC /TX', 'original_name': 'ExxonMobil'}
{'cik': '0000789019', 'name': 'MICROSOFT CORP', 'original_name': 'Microsoft'}
{'cik': '0000320193', 'name': 'Apple Inc.', 'original_name': 'Apple'}
{'cik': '0001095697', 'name': 'SHELL US GAS & POWER LLC', 'original_name': 'Shell'}
{'cik': '0001289018', 'name': 'INDUSTRIAL AND COMMERCIAL BANK OF CHINA FINANCIAL SERVICES LLC', 'original_name': 'Bank of China'}
{'cik': '0000834071', 'name': 'TOYOTA MOTOR CREDIT CORP', 'original_name': 'Toyota Motor'}
{'cik': '0000879316', 'name': 'SAMSUNG ELECTRONICS CO LTD /FI', 'original_name'

In [99]:
filtered_cik_list = []
for item in cik_list:
    if type(item) == dict:
        filtered_cik_list.append(item)

In [100]:
cik_df = pd.DataFrame.from_records(filtered_cik_list)
cik_df = cik_df.rename(columns={"original_name": "company_name"})
cik_df["company_name"] = cik_df["company_name"].astype(str)
cik_df

,cik,name,company_name
0,0000019617,JPMORGAN CHASE & CO,JPMorgan Chase
1,0000917358,ICBC STANDARD SECURITIES INC.,ICBC
2,0000070858,BANK OF AMERICA CORP /DE/,Bank of America
3,0001413936,ALPHABET PARTNERS LP,Alphabet
4,0001082970,EXXONMOBIL INVESTMENT MANAGEMENT INC /TX,ExxonMobil
...,...,...,...
154,0001045810,NVIDIA CORP,NVIDIA
155,0000066740,3M CO,3M
156,0000821189,EOG RESOURCES INC,EOG Resources
157,0001395064,TAKEDA PHARMACEUTICAL CO LTD,Takeda Pharmaceutical


In [101]:
merged_cik_df = pd.merge(df_2022_clean, cik_df, on="company_name", how="left")
merged_cik_df

,company_name,scope_1,scope_2,scope_3,sales_revenue,profit,data_source,year,cik,name
0,JPMorgan Chase,88 553,783 616,156 845,$179 930,$41 800,page 1: https://www.jpmorganchase.com/content/...,2022,0000019617,JPMORGAN CHASE & CO
1,Saudi Arabian Oil Company (Saudi Aramco),55 700 000,16 100 000,NaN,$589 470,$156 360,"page 63, https://www.aramco.com/-/media/public...",2022,NaN,NaN
2,ICBC,97 400,1 923 900,NaN,$216 770,$52 470,"page 27, https://v.icbc.com.cn/userfiles/Resou...",2022,0000917358,ICBC STANDARD SECURITIES INC.
3,China Construction Bank,69 497,1 613 315,NaN,$203 080,$48 250,"page 52, http://group.ccb.com/eng/attachDir/20...",2022,NaN,NaN
4,Agricultural Bank of China,222 353,1 931 555,NaN,$186 140,$37 920,"page 98, https://www.abchina.com/en/aboutus/cs...",2022,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
203,NVIDIA,4 612,133 569,2 701 477,$26 970,$4 370,"page no. 51, CSR 2022, https://images.nvidia.c...",2022,0001045810,NVIDIA CORP
204,3M,2 420 000,1 460 000,11 636 000,$33 430,$5 450,"page 149, https://multimedia.3m.com/mws/media/...",2022,0000066740,3M CO
205,EOG Resources,510 000,400 000,110 300 000,$28 430,$9 390,"page no. 4, https://eogresources-com.s3.us-wes...",2022,0000821189,EOG RESOURCES INC
206,Takeda Pharmaceutical,277 000,291 000,5 166 000,$29 990,$2 090,NaN,2022,0001395064,TAKEDA PHARMACEUTICAL CO LTD


In [104]:
merged_cik_df[~merged_cik_df['cik'].isna()].reset_index(drop=True)

,company_name,scope_1,scope_2,scope_3,sales_revenue,profit,data_source,year,cik,name
0,JPMorgan Chase,88 553,783 616,156 845,$179 930,$41 800,page 1: https://www.jpmorganchase.com/content/...,2022,0000019617,JPMORGAN CHASE & CO
1,ICBC,97 400,1 923 900,NaN,$216 770,$52 470,"page 27, https://v.icbc.com.cn/userfiles/Resou...",2022,0000917358,ICBC STANDARD SECURITIES INC.
2,Bank of America,66 775,634 510,3 023 784,$133 840,$28 620,"page 17, https://about.bankofamerica.com/conte...",2022,0000070858,BANK OF AMERICA CORP /DE/
3,Alphabet,91 200,8 045 400,7 600 000,$282 850,$58 590,page 90 https://www.gstatic.com/gumdrop/sustai...,2022,0001413936,ALPHABET PARTNERS LP
4,ExxonMobil,96 000 000,7 000 000,540 000 000,$393 160,$61 690,"page 3:, https://corporate.exxonmobil.com/-/me...",2022,0001082970,EXXONMOBIL INVESTMENT MANAGEMENT INC /TX
...,...,...,...,...,...,...,...,...,...,...
154,NVIDIA,4 612,133 569,2 701 477,$26 970,$4 370,"page no. 51, CSR 2022, https://images.nvidia.c...",2022,0001045810,NVIDIA CORP
155,3M,2 420 000,1 460 000,11 636 000,$33 430,$5 450,"page 149, https://multimedia.3m.com/mws/media/...",2022,0000066740,3M CO
156,EOG Resources,510 000,400 000,110 300 000,$28 430,$9 390,"page no. 4, https://eogresources-com.s3.us-wes...",2022,0000821189,EOG RESOURCES INC
157,Takeda Pharmaceutical,277 000,291 000,5 166 000,$29 990,$2 090,NaN,2022,0001395064,TAKEDA PHARMACEUTICAL CO LTD


In [106]:
ticker_list = []
for company_name in df_2022_clean['company_name']:
    formatted_company_name = company_name.strip(' ').replace(' ', '%20')

    response_obj = requests.get(f'https://financialmodelingprep.com/api/v3/search?query={formatted_company_name}&apikey=SWGIUgFhzgYfPKE5F9NsDyPX3SYTb72k')
    json_obj = response_obj.json()

    while type(json_obj) == list:
        if len(json_obj) > 0:
            json_obj = json_obj[0]
        else:
            break

    if type(json_obj) == dict:
        json_obj['original_name'] = company_name.strip(' ')

    ticker_list.append(json_obj)
    print(json_obj)

{'symbol': 'JPM-PM', 'name': 'JPMorgan Chase & Co.', 'currency': 'USD', 'stockExchange': 'New York Stock Exchange', 'exchangeShortName': 'NYSE', 'original_name': 'JPMorgan Chase'}
[]
{'symbol': '9011.HK', 'name': 'ICBC CICC USD Money Market ETF', 'currency': 'USD', 'stockExchange': 'HKSE', 'exchangeShortName': 'HKSE', 'original_name': 'ICBC'}
{'symbol': 'CNCB.VI', 'name': 'China Construction Bank Corporation', 'currency': 'EUR', 'stockExchange': 'Vienna', 'exchangeShortName': 'VIE', 'original_name': 'China Construction Bank'}
{'symbol': 'EK7.VI', 'name': 'Agricultural Bank of China Limited', 'currency': 'EUR', 'stockExchange': 'Vienna', 'exchangeShortName': 'VIE', 'original_name': 'Agricultural Bank of China'}
{'symbol': 'BAC.BA', 'name': 'Bank of America Corp', 'currency': 'USD', 'stockExchange': 'Buenos Aires', 'exchangeShortName': 'BUE', 'original_name': 'Bank of America'}
{'symbol': 'GOOG.NE', 'name': 'Alphabet Inc.', 'currency': 'CAD', 'stockExchange': 'NEO', 'exchangeShortName': 

In [107]:
filtered_ticker_list = []
for item in cik_list:
    if type(item) == dict:
        filtered_ticker_list.append(item)

In [110]:
ticker_df = pd.DataFrame.from_records(filtered_ticker_list)
ticker_df = ticker_df.rename(columns={"original_name": "company_name"})
ticker_df["company_name"] = ticker_df["company_name"].astype(str)
ticker_df

,symbol,name,currency,stockExchange,exchangeShortName,company_name
0,JPM-PM,JPMorgan Chase & Co.,USD,New York Stock Exchange,NYSE,JPMorgan Chase
1,9011.HK,ICBC CICC USD Money Market ETF,USD,HKSE,HKSE,ICBC
2,CNCB.VI,China Construction Bank Corporation,EUR,Vienna,VIE,China Construction Bank
3,EK7.VI,Agricultural Bank of China Limited,EUR,Vienna,VIE,Agricultural Bank of China
4,BAC.BA,Bank of America Corp,USD,Buenos Aires,BUE,Bank of America
...,...,...,...,...,...,...
183,NVD.F,NVIDIA Corporation,EUR,Frankfurt Stock Exchange,XETRA,NVIDIA
184,3MSF.L,Leverage Shares 3x Microsoft ETC,GBp,London Stock Exchange,LSE,3M
185,EOG,"EOG Resources, Inc.",USD,New York Stock Exchange,NYSE,EOG Resources
186,TKPHF,Takeda Pharmaceutical Company Limited,USD,Other OTC,PNK,Takeda Pharmaceutical


In [111]:
merged_ticker_df = pd.merge(merged_cik_df, ticker_df, on="company_name", how="left")
merged_ticker_df

,company_name,scope_1,scope_2,scope_3,sales_revenue,profit,data_source,year,cik,name_x,symbol,name_y,currency,stockExchange,exchangeShortName
0,JPMorgan Chase,88 553,783 616,156 845,$179 930,$41 800,page 1: https://www.jpmorganchase.com/content/...,2022,0000019617,JPMORGAN CHASE & CO,JPM-PM,JPMorgan Chase & Co.,USD,New York Stock Exchange,NYSE
1,Saudi Arabian Oil Company (Saudi Aramco),55 700 000,16 100 000,NaN,$589 470,$156 360,"page 63, https://www.aramco.com/-/media/public...",2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ICBC,97 400,1 923 900,NaN,$216 770,$52 470,"page 27, https://v.icbc.com.cn/userfiles/Resou...",2022,0000917358,ICBC STANDARD SECURITIES INC.,9011.HK,ICBC CICC USD Money Market ETF,USD,HKSE,HKSE
3,China Construction Bank,69 497,1 613 315,NaN,$203 080,$48 250,"page 52, http://group.ccb.com/eng/attachDir/20...",2022,NaN,NaN,CNCB.VI,China Construction Bank Corporation,EUR,Vienna,VIE
4,Agricultural Bank of China,222 353,1 931 555,NaN,$186 140,$37 920,"page 98, https://www.abchina.com/en/aboutus/cs...",2022,NaN,NaN,EK7.VI,Agricultural Bank of China Limited,EUR,Vienna,VIE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,NVIDIA,4 612,133 569,2 701 477,$26 970,$4 370,"page no. 51, CSR 2022, https://images.nvidia.c...",2022,0001045810,NVIDIA CORP,NVD.F,NVIDIA Corporation,EUR,Frankfurt Stock Exchange,XETRA
204,3M,2 420 000,1 460 000,11 636 000,$33 430,$5 450,"page 149, https://multimedia.3m.com/mws/media/...",2022,0000066740,3M CO,3MSF.L,Leverage Shares 3x Microsoft ETC,GBp,London Stock Exchange,LSE
205,EOG Resources,510 000,400 000,110 300 000,$28 430,$9 390,"page no. 4, https://eogresources-com.s3.us-wes...",2022,0000821189,EOG RESOURCES INC,EOG,"EOG Resources, Inc.",USD,New York Stock Exchange,NYSE
206,Takeda Pharmaceutical,277 000,291 000,5 166 000,$29 990,$2 090,NaN,2022,0001395064,TAKEDA PHARMACEUTICAL CO LTD,TKPHF,Takeda Pharmaceutical Company Limited,USD,Other OTC,PNK


In [ ]:
ticker_list = []
for company_name in df_2022_clean['company_name']:
    df_2022

    response_obj = requests.get(f'https://financialmodelingprep.com/api/v3/search?query={formatted_company_name}&apikey=SWGIUgFhzgYfPKE5F9NsDyPX3SYTb72k')
    json_obj = response_obj.json()

    while type(json_obj) == list:
        if len(json_obj) > 0:
            json_obj = json_obj[0]
        else:
            break

    if type(json_obj) == dict:
        json_obj['original_name'] = company_name.strip(' ')

    ticker_list.append(json_obj)
    print(json_obj)

In [144]:
merged_ticker_df

,company_name,scope_1,scope_2,scope_3,sales_revenue,profit,data_source,year,cik,name_x,symbol,name_y,currency,stockExchange,exchangeShortName
0,JPMorgan Chase,88 553,783 616,156 845,$179 930,$41 800,page 1: https://www.jpmorganchase.com/content/...,2022,0000019617,JPMORGAN CHASE & CO,JPM-PM,JPMorgan Chase & Co.,USD,New York Stock Exchange,NYSE
1,Saudi Arabian Oil Company (Saudi Aramco),55 700 000,16 100 000,NaN,$589 470,$156 360,"page 63, https://www.aramco.com/-/media/public...",2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ICBC,97 400,1 923 900,NaN,$216 770,$52 470,"page 27, https://v.icbc.com.cn/userfiles/Resou...",2022,0000917358,ICBC STANDARD SECURITIES INC.,9011.HK,ICBC CICC USD Money Market ETF,USD,HKSE,HKSE
3,China Construction Bank,69 497,1 613 315,NaN,$203 080,$48 250,"page 52, http://group.ccb.com/eng/attachDir/20...",2022,NaN,NaN,CNCB.VI,China Construction Bank Corporation,EUR,Vienna,VIE
4,Agricultural Bank of China,222 353,1 931 555,NaN,$186 140,$37 920,"page 98, https://www.abchina.com/en/aboutus/cs...",2022,NaN,NaN,EK7.VI,Agricultural Bank of China Limited,EUR,Vienna,VIE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,NVIDIA,4 612,133 569,2 701 477,$26 970,$4 370,"page no. 51, CSR 2022, https://images.nvidia.c...",2022,0001045810,NVIDIA CORP,NVD.F,NVIDIA Corporation,EUR,Frankfurt Stock Exchange,XETRA
204,3M,2 420 000,1 460 000,11 636 000,$33 430,$5 450,"page 149, https://multimedia.3m.com/mws/media/...",2022,0000066740,3M CO,3MSF.L,Leverage Shares 3x Microsoft ETC,GBp,London Stock Exchange,LSE
205,EOG Resources,510 000,400 000,110 300 000,$28 430,$9 390,"page no. 4, https://eogresources-com.s3.us-wes...",2022,0000821189,EOG RESOURCES INC,EOG,"EOG Resources, Inc.",USD,New York Stock Exchange,NYSE
206,Takeda Pharmaceutical,277 000,291 000,5 166 000,$29 990,$2 090,NaN,2022,0001395064,TAKEDA PHARMACEUTICAL CO LTD,TKPHF,Takeda Pharmaceutical Company Limited,USD,Other OTC,PNK


In [146]:
income_list = []
for index, row in merged_ticker_df.iterrows():
    if (type(row['cik']) == str):
        response_obj = requests.get(f"https://financialmodelingprep.com/api/v3/income-statement/{row['cik']}?period=annual&apikey=SWGIUgFhzgYfPKE5F9NsDyPX3SYTb72k")
        json_obj = response_obj.json()
    elif (type(row['symbol']) == str):
        response_obj = requests.get(f"https://financialmodelingprep.com/api/v3/income-statement/{row['symbol']}?period=annual&apikey=SWGIUgFhzgYfPKE5F9NsDyPX3SYTb72k")
        json_obj = response_obj.json()
    else:
        continue

    if type(json_obj) == list:
        for item in json_obj:
            if item['calendarYear'] == '2022':
                json_obj = item
                break

    if type(json_obj) == dict:
        json_obj['original_name'] = row['company_name']

    income_list.append(json_obj)
    print(f"symbol: {row['cik']}")
    print(f"cik: {row['cik']}")
    print(f"JSON: {json_obj}")
    print()
    
income_list

symbol: 0000019617
cik: 0000019617
JSON: {'date': '2022-12-31', 'symbol': 'JPM-PJ', 'reportedCurrency': 'USD', 'cik': '0000019617', 'fillingDate': '2023-02-21', 'acceptedDate': '2023-02-21 16:13:55', 'calendarYear': '2022', 'period': 'FY', 'revenue': 128695000000, 'costOfRevenue': 0, 'grossProfit': 128695000000, 'grossProfitRatio': 1, 'researchAndDevelopmentExpenses': 0, 'generalAndAdministrativeExpenses': 41636000000, 'sellingAndMarketingExpenses': 3911000000, 'sellingGeneralAndAdministrativeExpenses': 45547000000, 'otherExpenses': -109030000000, 'operatingExpenses': -63483000000, 'costAndExpenses': -63483000000, 'interestIncome': 92807000000, 'interestExpense': 26097000000, 'depreciationAndAmortization': 7051000000, 'ebitda': 72263000000, 'ebitdaratio': 0.561505886, 'operatingIncome': 65212000000, 'operatingIncomeRatio': 0.5067174327, 'totalOtherIncomeExpensesNet': -19046000000, 'incomeBeforeTax': 46166000000, 'incomeBeforeTaxRatio': 0.3587241152, 'incomeTaxExpense': 8490000000, 'net

[{'date': '2022-12-31',
  'symbol': 'JPM-PJ',
  'reportedCurrency': 'USD',
  'cik': '0000019617',
  'fillingDate': '2023-02-21',
  'acceptedDate': '2023-02-21 16:13:55',
  'calendarYear': '2022',
  'period': 'FY',
  'revenue': 128695000000,
  'costOfRevenue': 0,
  'grossProfit': 128695000000,
  'grossProfitRatio': 1,
  'researchAndDevelopmentExpenses': 0,
  'generalAndAdministrativeExpenses': 41636000000,
  'sellingAndMarketingExpenses': 3911000000,
  'sellingGeneralAndAdministrativeExpenses': 45547000000,
  'otherExpenses': -109030000000,
  'operatingExpenses': -63483000000,
  'costAndExpenses': -63483000000,
  'interestIncome': 92807000000,
  'interestExpense': 26097000000,
  'depreciationAndAmortization': 7051000000,
  'ebitda': 72263000000,
  'ebitdaratio': 0.561505886,
  'operatingIncome': 65212000000,
  'operatingIncomeRatio': 0.5067174327,
  'totalOtherIncomeExpensesNet': -19046000000,
  'incomeBeforeTax': 46166000000,
  'incomeBeforeTaxRatio': 0.3587241152,
  'incomeTaxExpense'

In [147]:
filtered_income_list = []
for item in income_list:
    if type(item) == dict:
        filtered_income_list.append(item)

filtered_income_list

[{'date': '2022-12-31',
  'symbol': 'JPM-PJ',
  'reportedCurrency': 'USD',
  'cik': '0000019617',
  'fillingDate': '2023-02-21',
  'acceptedDate': '2023-02-21 16:13:55',
  'calendarYear': '2022',
  'period': 'FY',
  'revenue': 128695000000,
  'costOfRevenue': 0,
  'grossProfit': 128695000000,
  'grossProfitRatio': 1,
  'researchAndDevelopmentExpenses': 0,
  'generalAndAdministrativeExpenses': 41636000000,
  'sellingAndMarketingExpenses': 3911000000,
  'sellingGeneralAndAdministrativeExpenses': 45547000000,
  'otherExpenses': -109030000000,
  'operatingExpenses': -63483000000,
  'costAndExpenses': -63483000000,
  'interestIncome': 92807000000,
  'interestExpense': 26097000000,
  'depreciationAndAmortization': 7051000000,
  'ebitda': 72263000000,
  'ebitdaratio': 0.561505886,
  'operatingIncome': 65212000000,
  'operatingIncomeRatio': 0.5067174327,
  'totalOtherIncomeExpensesNet': -19046000000,
  'incomeBeforeTax': 46166000000,
  'incomeBeforeTaxRatio': 0.3587241152,
  'incomeTaxExpense'

In [149]:
income_df = pd.DataFrame.from_records(filtered_income_list)
income_df

,date,symbol,reportedCurrency,cik,fillingDate,acceptedDate,calendarYear,period,revenue,costOfRevenue,...,incomeTaxExpense,netIncome,netIncomeRatio,eps,epsdiluted,weightedAverageShsOut,weightedAverageShsOutDil,link,finalLink,original_name
0,2022-12-31,JPM-PJ,USD,0000019617,2023-02-21,2023-02-21 16:13:55,2022,FY,128695000000,0,...,8490000000,37676000000,0.292754,12.47,12.47,2968734491,3021000000,https://www.sec.gov/Archives/edgar/data/19617/...,https://www.sec.gov/Archives/edgar/data/19617/...,JPMorgan Chase
1,2022-12-31,CNCB.VI,CNY,0000000000,2022-12-31,2022-12-30 19:00:00,2022,FY,821279000000,0,...,58851000000,324727000000,0.395392,1.28,1.28,253016406250,253016406250,,,China Construction Bank
2,2022-12-31,EK7.VI,CNY,0000000000,2022-12-31,2022-12-30 19:00:00,2022,FY,724868000000,0,...,47528000000,259232000000,0.357626,0.69,0.69,349983033873,349983033873,,,Agricultural Bank of China
3,2022-12-31,BAC-PM,USD,0000070858,2023-02-22,2023-02-22 16:33:50,2022,FY,94950000000,0,...,3441000000,27528000000,0.289921,0.00,0.00,0,0,https://www.sec.gov/Archives/edgar/data/70858/...,https://www.sec.gov/Archives/edgar/data/70858/...,Bank of America
4,2022-06-30,MSFT,USD,0000789019,2022-07-28,2022-07-28 16:06:19,2022,FY,198270000000,62650000000,...,10978000000,72738000000,0.366863,9.70,9.65,7496000000,7540000000,https://www.sec.gov/Archives/edgar/data/789019...,https://www.sec.gov/Archives/edgar/data/789019...,Microsoft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2022-01-30,NVDA,USD,0001045810,2022-03-18,2022-03-17 20:33:34,2022,FY,26914000000,9439000000,...,189000000,9752000000,0.362339,0.39,0.39,24960000000,25350000000,https://www.sec.gov/Archives/edgar/data/104581...,https://www.sec.gov/Archives/edgar/data/104581...,NVIDIA
131,2022-12-31,MMM,USD,0000066740,2023-02-08,2023-02-08 12:59:49,2022,FY,34229000000,19232000000,...,612000000,5777000000,0.168775,10.21,10.18,566000000,567600000,https://www.sec.gov/Archives/edgar/data/66740/...,https://www.sec.gov/Archives/edgar/data/66740/...,3M
132,2022-12-31,EOG,USD,0000821189,2023-02-23,2023-02-23 16:23:25,2022,FY,29492000000,5494000000,...,2142000000,7759000000,0.263088,13.31,13.22,583000000,587000000,https://www.sec.gov/Archives/edgar/data/821189...,https://www.sec.gov/Archives/edgar/data/821189...,EOG Resources
133,2022-03-31,TKPHF,JPY,0001395064,2022-06-29,2022-06-29 06:05:31,2022,FY,3569006000000,1106846000000,...,72405000000,230059000000,0.064460,147.14,145.87,1563501000,1577169000,https://www.sec.gov/Archives/edgar/data/139506...,https://www.sec.gov/Archives/edgar/data/139506...,Takeda Pharmaceutical


In [153]:
merged_ticker_df

,company_name,scope_1,scope_2,scope_3,sales_revenue,profit,data_source,year,cik,name_x,symbol,name_y,currency,stockExchange,exchangeShortName
0,JPMorgan Chase,88 553,783 616,156 845,$179 930,$41 800,page 1: https://www.jpmorganchase.com/content/...,2022,0000019617,JPMORGAN CHASE & CO,JPM-PM,JPMorgan Chase & Co.,USD,New York Stock Exchange,NYSE
1,Saudi Arabian Oil Company (Saudi Aramco),55 700 000,16 100 000,NaN,$589 470,$156 360,"page 63, https://www.aramco.com/-/media/public...",2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ICBC,97 400,1 923 900,NaN,$216 770,$52 470,"page 27, https://v.icbc.com.cn/userfiles/Resou...",2022,0000917358,ICBC STANDARD SECURITIES INC.,9011.HK,ICBC CICC USD Money Market ETF,USD,HKSE,HKSE
3,China Construction Bank,69 497,1 613 315,NaN,$203 080,$48 250,"page 52, http://group.ccb.com/eng/attachDir/20...",2022,NaN,NaN,CNCB.VI,China Construction Bank Corporation,EUR,Vienna,VIE
4,Agricultural Bank of China,222 353,1 931 555,NaN,$186 140,$37 920,"page 98, https://www.abchina.com/en/aboutus/cs...",2022,NaN,NaN,EK7.VI,Agricultural Bank of China Limited,EUR,Vienna,VIE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,NVIDIA,4 612,133 569,2 701 477,$26 970,$4 370,"page no. 51, CSR 2022, https://images.nvidia.c...",2022,0001045810,NVIDIA CORP,NVD.F,NVIDIA Corporation,EUR,Frankfurt Stock Exchange,XETRA
204,3M,2 420 000,1 460 000,11 636 000,$33 430,$5 450,"page 149, https://multimedia.3m.com/mws/media/...",2022,0000066740,3M CO,3MSF.L,Leverage Shares 3x Microsoft ETC,GBp,London Stock Exchange,LSE
205,EOG Resources,510 000,400 000,110 300 000,$28 430,$9 390,"page no. 4, https://eogresources-com.s3.us-wes...",2022,0000821189,EOG RESOURCES INC,EOG,"EOG Resources, Inc.",USD,New York Stock Exchange,NYSE
206,Takeda Pharmaceutical,277 000,291 000,5 166 000,$29 990,$2 090,NaN,2022,0001395064,TAKEDA PHARMACEUTICAL CO LTD,TKPHF,Takeda Pharmaceutical Company Limited,USD,Other OTC,PNK


In [155]:
income_df_clean = income_df.rename(columns={"original_name": "company_name"})
income_df_clean = income_df_clean[["company_name", "reportedCurrency", "revenue", "grossProfitRatio", "ebitda", "netIncome"]]
final_df = pd.merge(merged_ticker_df, income_df_clean, on="company_name", how="left")
final_df

,company_name,scope_1,scope_2,scope_3,sales_revenue,profit,data_source,year,cik,name_x,symbol,name_y,currency,stockExchange,exchangeShortName,reportedCurrency,revenue,grossProfitRatio,ebitda,netIncome
0,JPMorgan Chase,88 553,783 616,156 845,$179 930,$41 800,page 1: https://www.jpmorganchase.com/content/...,2022,0000019617,JPMORGAN CHASE & CO,JPM-PM,JPMorgan Chase & Co.,USD,New York Stock Exchange,NYSE,USD,1.286950e+11,1.000000,7.226300e+10,3.767600e+10
1,Saudi Arabian Oil Company (Saudi Aramco),55 700 000,16 100 000,NaN,$589 470,$156 360,"page 63, https://www.aramco.com/-/media/public...",2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ICBC,97 400,1 923 900,NaN,$216 770,$52 470,"page 27, https://v.icbc.com.cn/userfiles/Resou...",2022,0000917358,ICBC STANDARD SECURITIES INC.,9011.HK,ICBC CICC USD Money Market ETF,USD,HKSE,HKSE,NaN,NaN,NaN,NaN,NaN
3,China Construction Bank,69 497,1 613 315,NaN,$203 080,$48 250,"page 52, http://group.ccb.com/eng/attachDir/20...",2022,NaN,NaN,CNCB.VI,China Construction Bank Corporation,EUR,Vienna,VIE,CNY,8.212790e+11,1.000000,3.827120e+11,3.247270e+11
4,Agricultural Bank of China,222 353,1 931 555,NaN,$186 140,$37 920,"page 98, https://www.abchina.com/en/aboutus/cs...",2022,NaN,NaN,EK7.VI,Agricultural Bank of China Limited,EUR,Vienna,VIE,CNY,7.248680e+11,1.000000,8.455800e+11,2.592320e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,NVIDIA,4 612,133 569,2 701 477,$26 970,$4 370,"page no. 51, CSR 2022, https://images.nvidia.c...",2022,0001045810,NVIDIA CORP,NVD.F,NVIDIA Corporation,EUR,Frankfurt Stock Exchange,XETRA,USD,2.691400e+10,0.649290,1.135100e+10,9.752000e+09
204,3M,2 420 000,1 460 000,11 636 000,$33 430,$5 450,"page 149, https://multimedia.3m.com/mws/media/...",2022,0000066740,3M CO,3MSF.L,Leverage Shares 3x Microsoft ETC,GBp,London Stock Exchange,LSE,USD,3.422900e+10,0.438137,8.370000e+09,5.777000e+09
205,EOG Resources,510 000,400 000,110 300 000,$28 430,$9 390,"page no. 4, https://eogresources-com.s3.us-wes...",2022,0000821189,EOG RESOURCES INC,EOG,"EOG Resources, Inc.",USD,New York Stock Exchange,NYSE,USD,2.949200e+10,0.813712,1.528000e+10,7.759000e+09
206,Takeda Pharmaceutical,277 000,291 000,5 166 000,$29 990,$2 090,NaN,2022,0001395064,TAKEDA PHARMACEUTICAL CO LTD,TKPHF,Takeda Pharmaceutical Company Limited,USD,Other OTC,PNK,JPY,3.569006e+12,0.689873,1.052329e+12,2.300590e+11


In [156]:
final_df_clean = final_df.copy(deep=True)
final_df_clean = final_df_clean[["company_name", "scope_1", "scope_2", "scope_3", "sales_revenue", "profit", "revenue", "grossProfitRatio", "ebitda", "netIncome"]]
final_df_clean

,company_name,scope_1,scope_2,scope_3,sales_revenue,profit,revenue,grossProfitRatio,ebitda,netIncome
0,JPMorgan Chase,88 553,783 616,156 845,$179 930,$41 800,1.286950e+11,1.000000,7.226300e+10,3.767600e+10
1,Saudi Arabian Oil Company (Saudi Aramco),55 700 000,16 100 000,NaN,$589 470,$156 360,NaN,NaN,NaN,NaN
2,ICBC,97 400,1 923 900,NaN,$216 770,$52 470,NaN,NaN,NaN,NaN
3,China Construction Bank,69 497,1 613 315,NaN,$203 080,$48 250,8.212790e+11,1.000000,3.827120e+11,3.247270e+11
4,Agricultural Bank of China,222 353,1 931 555,NaN,$186 140,$37 920,7.248680e+11,1.000000,8.455800e+11,2.592320e+11
...,...,...,...,...,...,...,...,...,...,...
203,NVIDIA,4 612,133 569,2 701 477,$26 970,$4 370,2.691400e+10,0.649290,1.135100e+10,9.752000e+09
204,3M,2 420 000,1 460 000,11 636 000,$33 430,$5 450,3.422900e+10,0.438137,8.370000e+09,5.777000e+09
205,EOG Resources,510 000,400 000,110 300 000,$28 430,$9 390,2.949200e+10,0.813712,1.528000e+10,7.759000e+09
206,Takeda Pharmaceutical,277 000,291 000,5 166 000,$29 990,$2 090,3.569006e+12,0.689873,1.052329e+12,2.300590e+11


In [178]:
final_df_clean.to_json('../external_datasets/merged_api_data.json')

In [175]:
col_list = ['scope_1', 'scope_2', 'scope_3', 'sales_revenue',
       'profit']

def str_to_int(string):
    if type(string) == str:
        string = string.strip()
        string = string.strip('$')
        string = string.replace('\xa0', '')
        return int(string)

def clean_numeric_cols(col_list, df):
    df_copy = df.copy(deep=True)
    for col_name in col_list:
        df_copy[col_name] = df_copy[col_name].apply(str_to_int)
    df_copy = df_copy.fillna(0)
    return df_copy
    

In [177]:
final_df_clean = clean_numeric_cols(col_list, final_df_clean)
final_df_clean

,company_name,scope_1,scope_2,scope_3,sales_revenue,profit,revenue,grossProfitRatio,ebitda,netIncome
0,JPMorgan Chase,88553,783616.0,156845.0,179930,41800,1.286950e+11,1.000000,7.226300e+10,3.767600e+10
1,Saudi Arabian Oil Company (Saudi Aramco),55700000,16100000.0,0.0,589470,156360,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
2,ICBC,97400,1923900.0,0.0,216770,52470,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
3,China Construction Bank,69497,1613315.0,0.0,203080,48250,8.212790e+11,1.000000,3.827120e+11,3.247270e+11
4,Agricultural Bank of China,222353,1931555.0,0.0,186140,37920,7.248680e+11,1.000000,8.455800e+11,2.592320e+11
...,...,...,...,...,...,...,...,...,...,...
203,NVIDIA,4612,133569.0,2701477.0,26970,4370,2.691400e+10,0.649290,1.135100e+10,9.752000e+09
204,3M,2420000,1460000.0,11636000.0,33430,5450,3.422900e+10,0.438137,8.370000e+09,5.777000e+09
205,EOG Resources,510000,400000.0,110300000.0,28430,9390,2.949200e+10,0.813712,1.528000e+10,7.759000e+09
206,Takeda Pharmaceutical,277000,291000.0,5166000.0,29990,2090,3.569006e+12,0.689873,1.052329e+12,2.300590e+11


In [181]:
with pd.option_context('display.max_rows', 250, 'display.max_columns', 10):
    display(final_df_clean.head(250))

,company_name,scope_1,scope_2,scope_3,sales_revenue,profit,revenue,grossProfitRatio,ebitda,netIncome
0,JPMorgan Chase,88553,783616.0,1.568450e+05,179930,41800,1.286950e+11,1.000000,7.226300e+10,3.767600e+10
1,Saudi Arabian Oil Company (Saudi Aramco),55700000,16100000.0,0.000000e+00,589470,156360,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
2,ICBC,97400,1923900.0,0.000000e+00,216770,52470,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
3,China Construction Bank,69497,1613315.0,0.000000e+00,203080,48250,8.212790e+11,1.000000,3.827120e+11,3.247270e+11
4,Agricultural Bank of China,222353,1931555.0,0.000000e+00,186140,37920,7.248680e+11,1.000000,8.455800e+11,2.592320e+11
5,Bank of America,66775,634510.0,3.023784e+06,133840,28620,9.495000e+10,1.000000,5.305000e+10,2.752800e+10
6,Alphabet,91200,8045400.0,7.600000e+06,282850,58590,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
7,ExxonMobil,96000000,7000000.0,5.400000e+08,393160,61690,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
8,Microsoft,139413,6381250.0,1.634000e+07,207590,69020,1.982700e+11,0.684017,9.990500e+10,7.273800e+10
9,Apple,55200,3000.0,2.055000e+07,385100,94320,3.943280e+11,0.433096,1.305410e+11,9.980300e+10
